In [1]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git


Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495
Receiving objects: 100% (499/499), 3.07 MiB | 6.97 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [2]:
!pip install lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 6.0 MB/s eta 0:00:00


In [3]:
%cd deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


In [4]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

Downloading...
From: https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:01<00:00, 105MB/s]


In [5]:
!CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
demo_image/demo_1.png    	available                	0.9999
demo_image/demo_2.jpg    	shakeshack               	0.9530
demo_image/demo_3.png    	london               

In [6]:
!pip3 install fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=13ebaf103b30b042bbafb4e4044ee2da0761c4d777de02897b8d5f2000326be9
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [7]:
!unzip /content/drive/MyDrive/t/plate_img-train.zip -d /content/deep-text-recognition-benchmark/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/deep-text-recognition-benchmark/data/train/24244.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/train/24244.xml  
  inflating: /content/deep-text-recognition-benchmark/data/train/24245.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/train/24245.xml  
  inflating: /content/deep-text-recognition-benchmark/data/train/24249.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/train/24249.xml  
  inflating: /content/deep-text-recognition-benchmark/data/train/24250.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/train/24250.xml  
  inflating: /content/deep-text-recognition-benchmark/data/train/24251.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/train/24251.xml  
  inflating: /content/deep-text-recognition-benchmark/data/train/24252.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/train/24252.xml  
  inflating: /content/dee

In [8]:
!unzip /content/drive/MyDrive/t/plate_img-validation.zip -d /content/deep-text-recognition-benchmark/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/deep-text-recognition-benchmark/data/validation/02835.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02835.xml  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02845.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02845.xml  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02855.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02855.xml  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02865.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02865.xml  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02875.jpg  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02875.xml  
  inflating: /content/deep-text-recognition-benchmark/data/validation/02885.jpg  
  inflating: /content/deep-text-recognition-ben

In [9]:
!python3 create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_train.txt --outputPath dataset/train/

Written 1000 / 17466
Written 2000 / 17466
Written 3000 / 17466
Written 4000 / 17466
Written 5000 / 17466
Written 6000 / 17466
Written 7000 / 17466
Written 8000 / 17466
Written 9000 / 17466
Written 10000 / 17466
Written 11000 / 17466
Written 12000 / 17466
Written 13000 / 17466
Written 14000 / 17466
Written 15000 / 17466
Written 16000 / 17466
Written 17000 / 17466
Created dataset with 17466 samples


In [10]:
!python3 create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_validation.txt --outputPath dataset/validation/

Written 1000 / 2519
Written 2000 / 2519
Created dataset with 2519 samples


In [11]:
!python3 train.py \
--train_data dataset/train --valid_data dataset/validation \
--select_data / --batch_ratio 1 --num_iter 6000 --batch_max_length 8 --valInterval 100 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: dataset/train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    dataset/train	 dataset: /
sub-directory:	/.	 num samples: 17156
num total samples of /: 17156 x 1.0 (total_data_usage_ratio) = 17156
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slown